In [1]:
from ortools.linear_solver import pywraplp

# Define the constants
P = [7.5, 7, 10.7]  # Production per turbine in park 1, 2, 3
C_M = [100, 100, 100]  # Cost per turbine in park 1, 2, 3
N = 60  # Maximum number of turbines per park
L = 2  # Number of different cable types
C_C = [1.5, 2] #cable cost per km in million usd
S_I=[40, 25, 30] #length in km from shore to paprk (or park 1 to park 3)
C_L = []
for distance in S_I:
    cost_for_cable_from_park = [cost_per_km*distance for cost_per_km in C_C]
    C_L.append(cost_for_cable_from_park)

Q = [500, 900]  # Capacity of cables 1 and 2
D = 1300 # Required amount of delivered electricity
C_I = 2352  # Cost to establish the third wind farm

# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Variables
x = [solver.IntVar(0, N, f'x_{i+1}') for i in range(3)]  # x_1, x_2, x_3
delta = [
    [solver.BoolVar(f'delta_{i+1}_{ell+1}') for ell in range(L)] for i in range(3)
]  # delta_{i}{ell}

y = solver.BoolVar('y')  # Binary variable for the third park

# Constraints

# 1. Total production meets demand
solver.Add(sum(P[i] * x[i] for i in range(3)) >= D)

# 2. Upper limit on the number of turbines per park
for i in range(3):
    solver.Add(x[i] <= N)

# 3. Production in each park must be less than or equal to the capacity of the selected cable
for i in range(3):
    solver.Add(
        P[i] * x[i] <= sum(Q[ell] * delta[i][ell] for ell in range(L))
    )

# 4. Only one cable is used per park (modified for park 3)
for i in range(2):  # Parks 1 and 2
    solver.Add(sum(delta[i][ell] for ell in range(L)) == 1)
# For park 3, cable selection depends on whether the park is built
solver.Add(sum(delta[2][ell] for ell in range(L)) == y)

# Ensure that no cables are selected for park 3 when it's not built
for ell in range(L):
    solver.Add(delta[2][ell] <= y)

# 5. If the third park is not built, x_3 = 0
solver.Add(x[2] <= N * y)

# 6. Capacity constraint for parks 1 and 3 combined
solver.Add(
    P[0] * x[0] + P[2] * x[2]
    <= sum(Q[ell] * (delta[0][ell]) for ell in range(L)) # we have a bottle neck at park 1 because park 3 is connected via park 1
)

# 7. x_i are non-negative integers
for i in range(3):
    solver.Add(x[i] >= 0)
    
# # New constraint: at least one turbine in Park 1 and Park 2
# solver.Add(x[0] >= 1)  # Park 1 must have at least 1 turbine
# solver.Add(x[1] >= 1)  # Park 2 must have at least 1 turbine

# Objective function
objective = solver.Objective()

# Cost for turbines
for i in range(3):
    objective.SetCoefficient(x[i], C_M[i])

# Cost for cables
for i in range(3):
    for ell in range(L):
        objective.SetCoefficient(delta[i][ell], C_L[i][ell])

# Cost to establish the third park
objective.SetCoefficient(y, C_I)

objective.SetMinimization()

# Solve the problem
status = solver.Solve()

# Output the solution
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    for i in range(3):
        print(f'Number of turbines in park {i+1}: {x[i].solution_value()}')
    for i in range(3):
        for ell in range(L):
            if delta[i][ell].solution_value() > 0.5:
                print(f'Cable {ell+1} is used for park {i+1}')
    print(f'Is the third park built? {"Yes" if y.solution_value() > 0.5 else "No"}')
    
    print(f'Minimum total cost: {solver.Objective().Value()}')
    print()
    # print total production for each park
    for i in range(3):
        print(f'Total production in park {i+1}: {P[i] * x[i].solution_value()}')
else:
    print('The problem does not have an optimal solution.')


Solution:
Number of turbines in park 1: 32.0
Number of turbines in park 2: 60.0
Number of turbines in park 3: 60.0
Cable 2 is used for park 1
Cable 1 is used for park 2
Cable 2 is used for park 3
Is the third park built? Yes
Minimum total cost: 17729.5

Total production in park 1: 240.0
Total production in park 2: 420.0
Total production in park 3: 642.0
